In [11]:
import os
import pandas as pd
import librosa
import numpy as np
import soundfile as sf
from pedalboard import Pedalboard, Distortion, Reverb, Chorus, Delay  # Using Delay as Echo

# Define effect strength levels
effect_levels = [0.25, 0.5, 0.75, 1.0]

# Define effects with adjustable parameters (Fixed distortion: drive_db)
effects_dict = {
    "distortion": lambda level: Distortion(drive_db=level * 30),  # Drive in dB (scaled)
    "reverb": lambda level: Reverb(room_size=level),  # Room size
    "chorus": lambda level: Chorus(depth=level),  # Depth
    "echo": lambda level: Delay(delay_seconds=level * 0.3)  # Echo delay time
}

In [12]:
from os import listdir
from os.path import join

# Path to GuitarSet files
guitarset_path = "data/guitarset"
all_clean_guitar_files = [join(guitarset_path, f) for f in listdir(guitarset_path)]
# print(clean_guitar_files)
print(len(all_clean_guitar_files))

# Use first 100 for now
clean_guitar_files = all_clean_guitar_files[:360]
print(clean_guitar_files)

360
['data/guitarset/00_Jazz1-130-D_solo_mix.wav', 'data/guitarset/03_Funk2-119-G_comp_mix.wav', 'data/guitarset/04_Funk1-97-C_comp_mix.wav', 'data/guitarset/04_Jazz1-130-D_comp_mix.wav', 'data/guitarset/05_SS2-107-Ab_solo_mix.wav', 'data/guitarset/02_Jazz3-150-C_comp_mix.wav', 'data/guitarset/00_SS2-88-F_comp_mix.wav', 'data/guitarset/05_BN2-131-B_solo_mix.wav', 'data/guitarset/00_Rock2-142-D_comp_mix.wav', 'data/guitarset/02_Rock3-117-Bb_solo_mix.wav', 'data/guitarset/00_Funk2-108-Eb_solo_mix.wav', 'data/guitarset/02_Jazz3-137-Eb_comp_mix.wav', 'data/guitarset/01_Jazz2-110-Bb_solo_mix.wav', 'data/guitarset/04_Rock1-130-A_solo_mix.wav', 'data/guitarset/05_Jazz3-150-C_solo_mix.wav', 'data/guitarset/00_Rock3-117-Bb_solo_mix.wav', 'data/guitarset/02_Jazz1-200-B_solo_mix.wav', 'data/guitarset/05_BN3-119-G_comp_mix.wav', 'data/guitarset/02_Jazz2-187-F#_solo_mix.wav', 'data/guitarset/03_BN1-129-Eb_solo_mix.wav', 'data/guitarset/04_Funk2-108-Eb_solo_mix.wav', 'data/guitarset/01_SS1-100-C#_so

In [13]:
import os

# Directory to save processed audio files
output_dir = "data/generated"
os.makedirs(output_dir, exist_ok=True)

In [19]:
from tqdm import tqdm
import pyloudnorm as pyln

metadata = []

# Count total files to process (clean files × effects × levels)
total_files = len(clean_guitar_files) * len(effects_dict) * len(effect_levels)

# Initialize tqdm progress bar
progress_bar = tqdm(total=total_files, desc="Generating Processed Audio", unit="file")

# Iterate through clean guitar files
for clean_path in clean_guitar_files:
    # Load clean audio
    y_clean, sr = librosa.load(clean_path, sr=None)

    # Measure perceived loudness of the clean audio
    meter = pyln.Meter(sr)  # Create BS.1770 meter
    clean_loudness = meter.integrated_loudness(y_clean)

    # Get base filename
    base_filename = os.path.splitext(os.path.basename(clean_path))[0]

    # Process each effect at different levels
    for effect_name, effect_fn in effects_dict.items():
        for level in effect_levels:
            try:
                # Apply the effect with the specified level
                board = Pedalboard([effect_fn(level)])
                processed_audio = board(y_clean, sample_rate=sr)

                # Measure loudness of the processed audio
                processed_loudness = meter.integrated_loudness(processed_audio)

                # Normalize processed audio to match the clean audio loudness
                loudness_difference = clean_loudness - processed_loudness
                gain_factor = 10 ** (loudness_difference / 20)  # Convert dB difference to linear scale
                processed_audio = processed_audio * gain_factor  # Apply gain

                # Save processed file
                effect_filepath = os.path.join(output_dir, f"{base_filename}_{effect_name}_{level}.wav")
                sf.write(effect_filepath, processed_audio, sr)

                # Store metadata
                metadata.append({
                    "clean_file": clean_path,
                    "processed_file": effect_filepath,
                    "distortion_level": level if effect_name == "distortion" else 0.0,
                    "reverb_level": level if effect_name == "reverb" else 0.0,
                    "chorus_level": level if effect_name == "chorus" else 0.0,
                    "echo_level": level if effect_name == "echo" else 0.0
                })

                # Update tqdm progress bar
                progress_bar.update(1)

            except Exception as e:
                print(f"Error processing {clean_path} with {effect_name} at level {level}: {e}")
                continue

# Close progress bar
progress_bar.close()

# Convert metadata to DataFrame and save as CSV
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv(os.path.join(output_dir, "processed_audio_metadata.csv"), index=False)

print("Processed audio files saved with normalized loudness. Metadata stored in 'processed_audio_metadata.csv'.")

Generating Processed Audio:   1%|▏         | 86/5760 [00:10<11:28,  8.24file/s]


Generating Processed Audio: 100%|██████████| 5760/5760 [11:19<00:00,  8.47file/s]


Processed audio files saved with normalized loudness. Metadata stored in 'processed_audio_metadata.csv'.
